# Notebook for reading/plotting tensorboard logs

In [21]:
import pandas as pd
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator
import warnings


In [25]:
#choose which train folder we want
#with * we can have all of them and then choose one in the path variable, otherwise we choose only one folder
train_folder = 'basic_num_4'

In [26]:
#this gets all the logs in the folder
basedir = '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/{}/logs/'.format(train_folder)
logname = '*'

path = glob(os.path.expandvars(basedir + logname))

In [19]:
#function to get in a df all the variables from tensorboard
def load_tensorboard_df(path):
    ea = event_accumulator.EventAccumulator(path)
    ea.Reload()
    values = ea.Tags()['scalars']
    
    if not values:
        #p = path.split('/')
        #warnings.warn("The file {}, in the folder {}, does not contain any data".format(p[-1], p[-3]))
        return pd.DataFrame()
    else:
        #take one of them that share wall time and step to be the base
        df = pd.DataFrame(ea.Scalars(values[0]))[['wall_time', 'step']]

        #iterate over the values to get the data
        for val in values:
            colname = val.replace('/', '_')
            df = df.assign(**{colname : pd.DataFrame(ea.Scalars(val)).value})
        return df
    

In [20]:
load_tensorboard_df(path[0]) #the [0] is because the variable path was a list, not a string

,wall_time,step,loss_train,loss_valid,iou_train_0class,iou_train_1class,iou_train_2class,iou_valid_0class,iou_valid_1class,iou_valid_2class
0,1.656079e+09,0,0.036732,0.030367,0.245209,0.868742,0.359715,0.317365,0.882729,0.429303
1,1.656081e+09,1,0.029327,0.027768,0.333516,0.884234,0.461004,0.348624,0.887249,0.483223
2,1.656083e+09,2,0.027205,0.026250,0.361953,0.887623,0.490622,0.380796,0.888321,0.501682
3,1.656085e+09,3,0.025798,0.025014,0.380292,0.889846,0.507900,0.392422,0.891363,0.516099
4,1.656087e+09,4,0.024857,0.024241,0.393323,0.891501,0.518420,0.404002,0.893479,0.532448
5,1.656089e+09,5,0.024250,0.023748,0.403004,0.892883,0.526206,0.402680,0.893993,0.537473
6,1.656091e+09,6,0.023844,0.023479,0.411659,0.893932,0.530725,0.421139,0.895198,0.536304
7,1.656093e+09,7,0.023498,0.023247,0.420004,0.895058,0.534659,0.415551,0.895733,0.542090
8,1.656095e+09,8,0.023263,0.022902,0.426701,0.895823,0.537743,0.443928,0.897254,0.547303
9,1.656097e+09,9,0.023033,0.022702,0.431154,0.896487,0.540034,0.440958,0.898343,0.545809


We can also join all the information of all the tensorboards in one dataframe to get general stats

In [28]:
#choose which train folder we want
#with * we can have all of them and then choose one in the path variable, otherwise we choose only one folder
train_folder = '*'

In [29]:
#this gets all the logs in the folder
basedir = '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/{}/logs/'.format(train_folder)
logname = '*'

paths = glob(os.path.expandvars(basedir + logname))

Having all the paths to all the tb, we join them

In [30]:
paths

['/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/2_downsample/logs/events.out.tfevents.1655538381.mlwn17.ific.uv.es.133382.0',
 '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/basic_num_1/logs/events.out.tfevents.1655468864.mlwn05.ific.uv.es.168440.0',
 '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/basic_num_1/logs/events.out.tfevents.1655467603.mlwn17.ific.uv.es.31454.0',
 '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/basic_num_1/logs/events.out.tfevents.1655468872.mlwn01.ific.uv.es.134608.0',
 '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/basic_num_1/logs/events.out.tfevents.1655468192.mlwn17.ific.uv.es.32274.0',
 '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/prueba/logs/events.out.tfevents.1656339753.mlwn18.ific.uv.es.158047.0',
 '/lustre/ific.uv.es/ml/ific020/nexus_2021/segmentation/marcos_trains/nplanes_2/logs/events.out.tfevents.1655468823.mlwn16.i

This creates a df with all the values in the marcos_trains folder

In [31]:
data_df = pd.DataFrame()
for p in paths:
    spl = p.split('/')
    folder, file = spl[-3], spl[-1]
    ind_df = load_tensorboard_df(p)
    if not ind_df.empty:
        ind_df = ind_df.assign(**{'folder':spl[-3], 'file':spl[-1]})
        data_df = data_df.append(ind_df)
    else:
        print('The file {} in the folder {} was empty'.format(file, folder))

The file events.out.tfevents.1655467603.mlwn17.ific.uv.es.31454.0 in the folder basic_num_1 was empty
The file events.out.tfevents.1655468192.mlwn17.ific.uv.es.32274.0 in the folder basic_num_1 was empty
The file events.out.tfevents.1655468236.mlwn05.ific.uv.es.167588.0 in the folder basic_num_2 was empty
The file events.out.tfevents.1655467519.mlwn16.ific.uv.es.194415.0 in the folder basic_num_2 was empty


In [32]:
data_df

,wall_time,step,loss_train,loss_valid,iou_train_0class,iou_train_1class,iou_train_2class,iou_valid_0class,iou_valid_1class,iou_valid_2class,folder,file
0,1.655540e+09,0,0.037226,0.031323,0.235731,0.866552,0.337586,0.291571,0.879617,0.434975,2_downsample,events.out.tfevents.1655538381.mlwn17.ific.uv....
1,1.655542e+09,1,0.030280,0.029076,0.316000,0.882036,0.448350,0.325599,0.884779,0.472528,2_downsample,events.out.tfevents.1655538381.mlwn17.ific.uv....
2,1.655544e+09,2,0.028766,0.028181,0.338583,0.885305,0.474342,0.342811,0.886355,0.490788,2_downsample,events.out.tfevents.1655538381.mlwn17.ific.uv....
3,1.655546e+09,3,0.027859,0.027080,0.349388,0.886541,0.485810,0.356033,0.887460,0.501501,2_downsample,events.out.tfevents.1655538381.mlwn17.ific.uv....
4,1.655548e+09,4,0.027068,0.026380,0.358585,0.887267,0.493983,0.367382,0.888557,0.503179,2_downsample,events.out.tfevents.1655538381.mlwn17.ific.uv....
...,...,...,...,...,...,...,...,...,...,...,...,...
15,1.655563e+09,15,0.022853,0.022756,0.429542,0.896358,0.541776,0.436917,0.896751,0.548798,basic_num_2,events.out.tfevents.1655537326.mlwn07.ific.uv....
16,1.655564e+09,16,0.022769,0.022760,0.430392,0.896445,0.543022,0.425657,0.897108,0.549235,basic_num_2,events.out.tfevents.1655537326.mlwn07.ific.uv....
17,1.655566e+09,17,0.022714,0.022659,0.431983,0.896577,0.543091,0.425975,0.897167,0.543565,basic_num_2,events.out.tfevents.1655537326.mlwn07.ific.uv....
18,1.655567e+09,18,0.022647,0.022511,0.433898,0.896816,0.543230,0.432720,0.897513,0.551643,basic_num_2,events.out.tfevents.1655537326.mlwn07.ific.uv....
